In [4]:
%pip install transformers
%pip install -U datasets
    #ransformers \
    #atasets  --quiet

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/510.5 kB ? eta -:--:--
   ------ --------------------------------- 81.9/510.5 kB 2.3 MB/s eta 0:00:01
   ------ --------------------------------- 81.9/510.5 kB 2.3 MB/s eta 0:00:01
   ------- ------------------------------- 92.2/510.5 kB 744.7 kB/s eta 0:00:01
   ---------- --------------------------- 143.4/510.5 kB 853.3 kB/s eta 0:00:01
   -------------- ----------------------- 194.6/510.5 kB 908.0 kB/s eta 0:00:01
   -------------------------------- ------- 409.6/510.5 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 510.5/510.5 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/170.9 kB ? eta -:--:--
   ---------------------------------------- 170.9/170.9 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/24.8 MB 6.9 MB/s eta 0:00:04
    ---

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.2.0 which is incompatible.


In [22]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
from transformers import BertTokenizer, AutoModel, EncoderDecoderModel


In [3]:
huggingface_dataset_name = "csebuetnlp/xlsum"
dataset = load_dataset(huggingface_dataset_name,'indonesian')

Generating train split:   0%|          | 0/38242 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4780 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4780 [00:00<?, ? examples/s]

In [4]:
example_indices =[40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['text'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
Penjaga pantai berupaya untuk menyelamatkan korban, tujuh orang dilarikan ke rumah sakit Penjara pantai Korea Selatan mengatakan dua orang masih hilang, dan upaya pencarian masih dilakukan. Kapal memancing sewaan, Seonchang-1, membawa 20 penumpang dan dua kru dalam sebuah tur memancing saat tabrakan terjadi. Cuplikan gambar dari lokasi kejadian menunjukkan para penyelam sedang mencari korban perahu yang terbalik. Helikopter Angkatan Laut dan puluhan kapal lain ikut dalam pencarian di Incheon, yang terletak dekat pulau Yeongheung. Tujuh orang dibawa ke rumah sakit untuk mendapatkan perawatan. Kapten kapal memancing berukuran 10 ton itu masih hilang, menurut salah satu laporan AFP. Tidak ada laporan korban luka dari kru kapan tanker dengan ukuran 336-ton tersebut. K

In [23]:
model_name='cahya/bert2gpt-indonesian-summarization'

model = EncoderDecoderModel.from_pretrained(model_name)


pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

In [24]:
tokenizer = BertTokenizer.from_pretrained(model_name, use_fast=True)

In [25]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True)

print('Encoded Sentence:')
print(sentence_encoded["input_ids"][0])
print('Decoded Sentence:')
print(sentence_decoded)

Encoded Sentence:
tensor([    3, 12462,  6900,  1744,  2623,    15,  4257,    32,     1])
Decoded Sentence:
what time is it, tom?


In [26]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['text']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=200,
            )[0],
        skip_special_tokens=True)
    
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Penjaga pantai berupaya untuk menyelamatkan korban, tujuh orang dilarikan ke rumah sakit Penjara pantai Korea Selatan mengatakan dua orang masih hilang, dan upaya pencarian masih dilakukan. Kapal memancing sewaan, Seonchang-1, membawa 20 penumpang dan dua kru dalam sebuah tur memancing saat tabrakan terjadi. Cuplikan gambar dari lokasi kejadian menunjukkan para penyelam sedang mencari korban perahu yang terbalik. Helikopter Angkatan Laut dan puluhan kapal lain ikut dalam pencarian di Incheon, yang terletak dekat pulau Yeongheung. Tujuh orang dibawa ke rumah sakit untuk mendapatkan perawatan. Kapten kapal memancing berukuran 10 ton itu masih hilang, menurut salah satu laporan AFP. Tidak ada laporan korban luka dari kru kapan tanker dengan ukuran 336-ton tersebut. Kan

In [27]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['text']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
    Summarize the following conversation
    {dialogue}
    Summary:
    """
    
    
    
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True)
    
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - NO PROMPT ENGINEERING:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    Summarize the following conversation
    Penjaga pantai berupaya untuk menyelamatkan korban, tujuh orang dilarikan ke rumah sakit Penjara pantai Korea Selatan mengatakan dua orang masih hilang, dan upaya pencarian masih dilakukan. Kapal memancing sewaan, Seonchang-1, membawa 20 penumpang dan dua kru dalam sebuah tur memancing saat tabrakan terjadi. Cuplikan gambar dari lokasi kejadian menunjukkan para penyelam sedang mencari korban perahu yang terbalik. Helikopter Angkatan Laut dan puluhan kapal lain ikut dalam pencarian di Incheon, yang terletak dekat pulau Yeongheung. Tujuh orang dibawa ke rumah sakit untuk mendapatkan perawatan. Kapten kapal memancing berukuran 10 ton itu masih hilang, menurut salah satu laporan AFP. Tidak ada laporan korban luka dari kru ka

In [29]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['text']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
    Summarize the following conversation
    {dialogue}
    
    What was going on?
    """
    
    
    
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True)
    
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    Summarize the following conversation
    Penjaga pantai berupaya untuk menyelamatkan korban, tujuh orang dilarikan ke rumah sakit Penjara pantai Korea Selatan mengatakan dua orang masih hilang, dan upaya pencarian masih dilakukan. Kapal memancing sewaan, Seonchang-1, membawa 20 penumpang dan dua kru dalam sebuah tur memancing saat tabrakan terjadi. Cuplikan gambar dari lokasi kejadian menunjukkan para penyelam sedang mencari korban perahu yang terbalik. Helikopter Angkatan Laut dan puluhan kapal lain ikut dalam pencarian di Incheon, yang terletak dekat pulau Yeongheung. Tujuh orang dibawa ke rumah sakit untuk mendapatkan perawatan. Kapten kapal memancing berukuran 10 ton itu masih hilang, menurut salah satu laporan AFP. Tidak ada laporan korban luka dari kru ka

One-Shot Inference

In [30]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt =''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['text']
        summary = dataset['test'][index]['summary']
        
        prompt +=f"""
Dialogue:
{dialogue}
What was going on?
{summary}
"""
    dialogue = dataset['test'][example_index_to_summarize]['text']
    
    prompt +=f"""
Dialogue:
{dialogue}
What was going on?
"""
    return prompt
    

In [31]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:
Penjaga pantai berupaya untuk menyelamatkan korban, tujuh orang dilarikan ke rumah sakit Penjara pantai Korea Selatan mengatakan dua orang masih hilang, dan upaya pencarian masih dilakukan. Kapal memancing sewaan, Seonchang-1, membawa 20 penumpang dan dua kru dalam sebuah tur memancing saat tabrakan terjadi. Cuplikan gambar dari lokasi kejadian menunjukkan para penyelam sedang mencari korban perahu yang terbalik. Helikopter Angkatan Laut dan puluhan kapal lain ikut dalam pencarian di Incheon, yang terletak dekat pulau Yeongheung. Tujuh orang dibawa ke rumah sakit untuk mendapatkan perawatan. Kapten kapal memancing berukuran 10 ton itu masih hilang, menurut salah satu laporan AFP. Tidak ada laporan korban luka dari kru kapan tanker dengan ukuran 336-ton tersebut. Kantor berita Korea Selatan Yonhap mengatakan tabrakan terjadi sembilan menit setelah kapal berangkat dari pantai, kemungkinan ketika dua kapal itu bertemu di bawah sebuah jembatan. "Tidak ada masalah dengan kondisi 

In [38]:
summary = dataset['test'][example_index_to_summarize]['text']

inputs = tokenizer(one_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 100,
            )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'INPUT PROMPT:\n{one_shot_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOTS:\n{outputs}')


RuntimeError: The expanded size of the tensor (525) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 525].  Tensor sizes: [1, 512]

Few shot inference

In [35]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:
Penjaga pantai berupaya untuk menyelamatkan korban, tujuh orang dilarikan ke rumah sakit Penjara pantai Korea Selatan mengatakan dua orang masih hilang, dan upaya pencarian masih dilakukan. Kapal memancing sewaan, Seonchang-1, membawa 20 penumpang dan dua kru dalam sebuah tur memancing saat tabrakan terjadi. Cuplikan gambar dari lokasi kejadian menunjukkan para penyelam sedang mencari korban perahu yang terbalik. Helikopter Angkatan Laut dan puluhan kapal lain ikut dalam pencarian di Incheon, yang terletak dekat pulau Yeongheung. Tujuh orang dibawa ke rumah sakit untuk mendapatkan perawatan. Kapten kapal memancing berukuran 10 ton itu masih hilang, menurut salah satu laporan AFP. Tidak ada laporan korban luka dari kru kapan tanker dengan ukuran 336-ton tersebut. Kantor berita Korea Selatan Yonhap mengatakan tabrakan terjadi sembilan menit setelah kapal berangkat dari pantai, kemungkinan ketika dua kapal itu bertemu di bawah sebuah jembatan. "Tidak ada masalah dengan kondisi 

In [37]:
summary = dataset['test'][example_index_to_summarize]['text']

inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 100,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOTS:\n{outputs}')


RuntimeError: The expanded size of the tensor (1332) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 1332].  Tensor sizes: [1, 512]